In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import anndata
#import scanpy as sc

#sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
#sc.logging.print_versions()
#results_file = './write/paul15.h5ad'

In [ ]:
import anndata

In [ ]:
import cmocean

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import manifold
from collections import defaultdict
#from mpl_toolkits.mplot3d import Axes3D
import math
import re
import os
import sys
#imports
import pandas as pd
import pickle as pickle
from scipy.spatial.distance import cdist, pdist, squareform
#import backspinpy
#from backspinpy import fit_CV
#from backspinpy.Cef_tools import *
#from __future__ import division
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import StratifiedShuffleSplit  
from collections import defaultdict
from sklearn import preprocessing
import matplotlib.patches as mpatches
%pylab inline

In [ ]:
def fit_CV(mu, cv, fit_method='Exp', svr_gamma=0.06, x0=[0.5,0.5], verbose=False):
    '''Fits a noise model (CV vs mean)
    Parameters
    ----------
    mu: 1-D array
        mean of the genes (raw counts)
    cv: 1-D array
        coefficient of variation for each gene
    fit_method: string
        allowed: 'SVR', 'Exp', 'binSVR', 'binExp' 
        default: 'SVR'(requires scikit learn)
        SVR: uses Support vector regression to fit the noise model
        Exp: Parametric fit to cv = mu^(-a) + b
        bin: before fitting the distribution of mean is normalized to be
             uniform by downsampling and resampling.
    Returns
    -------
    score: 1-D array
        Score is the relative position with respect of the fitted curve
    mu_linspace: 1-D array
        x coordiantes to plot (min(log2(mu)) -> max(log2(mu)))
    cv_fit: 1-D array
        y=f(x) coordinates to plot 
    pars: tuple or None
    
    '''
    log2_m = log2(mu)
    log2_cv = log2(cv)
    
    if len(mu)>1000 and 'bin' in fit_method:
        #histogram with 30 bins
        n,xi = histogram(log2_m,30)
        med_n = percentile(n,50)
        for i in range(0,len(n)):
            # index of genes within the ith bin
            ind = where( (log2_m >= xi[i]) & (log2_m < xi[i+1]) )[0].astype(int)
            if len(ind)>med_n:
                #Downsample if count is more than median
                ind = ind[random.permutation(len(ind))]
                ind = ind[:len(ind)-int(med_n)]
                mask = ones(len(log2_m), dtype=bool)
                mask[ind] = False
                log2_m = log2_m[mask]
                log2_cv = log2_cv[mask]
            elif (around(med_n/len(ind))>1) and (len(ind)>5):
                #Duplicate if count is less than median
                log2_m = r_[ log2_m, tile(log2_m[ind], int(round(med_n/len(ind))-1)) ]
                log2_cv = r_[ log2_cv, tile(log2_cv[ind], int(round(med_n/len(ind))-1)) ]
    else:
        if 'bin' in fit_method:
            print('More than 1000 input feature needed for bin correction.')
        pass
                
    if 'SVR' in fit_method:
        try:
            from sklearn.svm import SVR
            if svr_gamma == 'auto':
                svr_gamma = 1000./len(mu)
            #Fit the Support Vector Regression
            clf = SVR(gamma=svr_gamma)
            clf.fit(log2_m[:,newaxis], log2_cv)
            fitted_fun = clf.predict
            score = log2(cv) - fitted_fun(log2(mu)[:,newaxis])
            params = None
            #The coordinates of the fitted curve
            mu_linspace = linspace(min(log2_m),max(log2_m))
            cv_fit = fitted_fun(mu_linspace[:,newaxis])
            return score, mu_linspace, cv_fit , params
            
        except ImportError:
            if verbose:
                print('SVR fit requires scikit-learn python library. Using exponential instead.')
            if 'bin' in fit_method:
                return fit_CV(mu, cv, fit_method='binExp', x0=x0)
            else:
                return fit_CV(mu, cv, fit_method='Exp', x0=x0)
    elif 'Exp' in fit_method:
        from scipy.optimize import minimize
        #Define the objective function to fit (least squares)
        fun = lambda x, log2_m, log2_cv: sum(abs( log2( (2.**log2_m)**(-x[0])+x[1]) - log2_cv ))
        #Fit using Nelder-Mead algorythm
        optimization =  minimize(fun, x0, args=(log2_m,log2_cv), method='Nelder-Mead')
        params = optimization.x
        #The fitted function
        fitted_fun = lambda log_mu: log2( (2.**log_mu)**(-params[0]) + params[1])
        # Score is the relative position with respect of the fitted curve
        score = log2(cv) - fitted_fun(log2(mu))
        #The coordinates of the fitted curve
        mu_linspace = linspace(min(log2_m),max(log2_m))
        cv_fit = fitted_fun(mu_linspace)
        return score, mu_linspace, cv_fit , params
    

In [ ]:
from scipy.optimize import curve_fit
from scipy.stats.distributions import  t

def func(x, a, b,c):
    'nonlinear function in a and b to fit to data'
    return a*x*x+b*x+c
def thrscount(x,y):
    initial_guess = [10, 0-x[-1], y[-1]]
    pars, pcov = curve_fit(func, x, y, p0=initial_guess)
    alpha = 0.05 # 95% confidence interval = 100*(1-alpha)

    n = len(y)    # number of data points
    p = len(pars) # number of parameters

    dof = max(0, n - p) # number of degrees of freedom

    # student-t value for the dof and confidence level
    tval = t.ppf(1.0-alpha/2., dof)
    for i, j,var in zip(range(n), pars, np.diag(pcov)):
        sigma = var**0.5
        print ('p{0}: {1} [{2}  {3}]'.format(i, j,
                                  j - sigma*tval,
                                  j + sigma*tval))
    return pars

In [ ]:
cd F:\Dropbox\data\proj\PE_HYZ\PublicDataSet\NBL_SZscRNA\SWAPLINE2

In [ ]:
dfp=pd.read_table("Final_scCAMEL_spatialData_MachineLearning_SpatialTumorCells_probability.tab",index_col=0,header=0,sep="\t")

In [ ]:
dfp.groupby(["sigType"]).count()

In [ ]:
dfp.head()

In [ ]:
cd F:\Dropbox\data\proj\PE_HYZ\PublicDataSet\NBL_SZscRNA\All_Dataset_Summary

In [ ]:
scpdt=anndata.read("VisiumSpatial4patients.h5ad")

In [ ]:
scpdt.obs.index=scpdt.obs["CellID"]

In [ ]:
scpdt.obs

In [ ]:
scpdt.obs["TumorCluster"]="Non-tumor"
scpdt.obs["TumorCluster"]=scpdt.obs["Cluster"]

scpdt.obs["Type"]="Non-tumor"
scpdt.obs.loc[dfp.index,"Type"]="Tumor"

In [ ]:
scpdt.obs

In [ ]:
#scpdt.obs["TumorCluster"].values==dfp["sigType"].values

In [ ]:
dftemp=scpdt.obs[[ 'T-cells', 'B-cells', 'Macrophages', 'Endothelial cells', 'DCs', 'CAFs']]
dftemp.index=scpdt.obs['CellID']
dftemp

In [ ]:
dftemp1=dftemp.loc[dftemp["T-cells"]>0]
dftemp1["sigType"]=["T-cells"]*dftemp1.shape[0]
dfprobnormal=dftemp1

In [ ]:
dfprobnormal

In [ ]:
for item in dftemp.columns[1:]:
    dftemp1=dftemp.loc[dftemp[item]>0]
    dftemp1["sigType"]=[item]*dftemp1.shape[0]
    dfprobnormal=dfprobnormal.append(dftemp1)

In [ ]:
dfprobnormal.shape

In [ ]:
set(dfprobnormal["sigType"])

In [ ]:
sum(dfprobnormal["T-cells"]>0)

In [ ]:
cd F:\Dropbox\data\proj\PE_HYZ\PublicDataSet\NBL_SZscRNA\SWAPLINE2

In [ ]:
dfprobtumor=scpdt.obs.loc[scpdt.obs["Type"]=="Tumor"]
dfprobtumor["sigType"]=dfprobtumor["Cluster"]

In [ ]:
dfprobtumor.groupby(["sigType"]).count()

In [ ]:
scpdt.obs.groupby(["TumorCluster"]).count()

In [ ]:
dfprobtumor.head()

In [ ]:
sum(dfprobnormal.iloc[:10,:-1].T>0,1).values

In [ ]:
dfprobtumor

In [ ]:
#scpdt.file.close()

In [ ]:
scpdt=anndata.read("szNBLSpatial_all4slides.h5ad")
scpdt

In [ ]:
cd F:\Dropbox\data\proj\PE_HYZ\PublicDataSet\NBL_SZscRNA\SWAPLINE2

# START

# For803

In [ ]:
dfposi=pd.read_table("B803_spot-spotDistance.tab",index_col=0,header=0,sep="\t")
dfposi.shape

In [ ]:
dfposi.head()

In [ ]:
dftumor=dfprobtumor.loc[dfprobtumor["Patient"]=="B803"].T
dftumor

In [ ]:
dfprobnormal

In [ ]:
dfprobnormal

In [ ]:
dfnormal=dfprobnormal.iloc[:,:-1].T
dfnormal

In [ ]:
dfn2=dfnormal.T
dfn2.shape

In [ ]:
set(dfn2.loc[dfn2["Macrophages"]>0].index) 

In [ ]:
len(set(dfn2.loc[dfn2["Macrophages"]>0].index) & set(dfposi.columns))

In [ ]:
dftumor.columns

In [ ]:
typelist=[]
for j in range(dftumor.shape[1]):
    dftemp=dfnormal[set(dfnormal.columns)&set(dfposi.loc[dfposi[dftumor.columns[j]]<=100].index)]
    typelist.append(sum(dftemp>0,1).values)

In [ ]:
dftype=pd.DataFrame(typelist)

In [ ]:
dftype.shape

In [ ]:
dftumor.shape

In [ ]:
dftype.columns=dfnormal.index
dftype.index=dftumor.columns
dftype

In [ ]:
dftumor

In [ ]:
swpdt=scpdt[scpdt.obs["Patient"]=="B803",:]
swpdt

In [ ]:
scpdt.obs.index=scpdt.obs["CellID"]

In [ ]:
dftype=dftype.join(scpdt.obs["TumorCluster"],how="inner")
dftype

In [ ]:
dftype.columns

In [ ]:
dftype.columns=['T-cells', 'B-cells', 'Macrophages', 'Endothelial cells', 'DCs',
       'CAFs', 'TumorCluster']

In [ ]:
dftype.groupby(["TumorCluster"]).sum()

In [ ]:
dftype.groupby(["TumorCluster"]).mean()

In [ ]:
dftype803=dftype

In [ ]:
##Merge

In [ ]:
dftype=dftype803.append(dftypeA254).append(dftypeNBLC1).append(dftypeNBLA1)

In [ ]:
dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["Macrophages"].values

In [ ]:
from scipy import stats

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["Macrophages"].values
YY=dftype.loc[dftype["TumorCluster"]=="Connecting"]["Macrophages"].values
[stats.ttest_ind(XX,YY, equal_var=False),XX.mean()/YY.mean()]

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["Macrophages"].values
YY=dftype.loc[dftype["TumorCluster"]=="NC_prog"]["Macrophages"].values
[stats.ttest_ind(XX,YY, equal_var=False),XX.mean()/YY.mean()]

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["Macrophages"].values
YY=dftype.loc[dftype["TumorCluster"]=="Diff_Mes"]["Macrophages"].values
[stats.ttest_ind(XX,YY, equal_var=False),XX.mean()/YY.mean()]

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["Macrophages"].values
YY=dftype.loc[dftype["TumorCluster"]=="preNeuroblast"]["Macrophages"].values
stats.ttest_ind(XX,YY, equal_var=False)

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["Macrophages"].values
YY=dftype.loc[dftype["TumorCluster"]=="Diff_Neuroblast"]["Macrophages"].values
[stats.ttest_ind(XX,YY, equal_var=False),XX.mean()/YY.mean()]

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["CAFs"].values
YY=dftype.loc[dftype["TumorCluster"]=="Connecting"]["CAFs"].values
[stats.ttest_ind(XX,YY, equal_var=False),XX.mean()/YY.mean()]

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["CAFs"].values
YY=dftype.loc[dftype["TumorCluster"]=="NC_prog"]["CAFs"].values
[stats.ttest_ind(XX,YY, equal_var=False),XX.mean()/YY.mean()]

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["CAFs"].values
YY=dftype.loc[dftype["TumorCluster"]=="Diff_Mes"]["CAFs"].values
[stats.ttest_ind(XX,YY, equal_var=False),XX.mean()/YY.mean()]

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["CAFs"].values
YY=dftype.loc[dftype["TumorCluster"]=="preNeuroblast"]["CAFs"].values
[stats.ttest_ind(XX,YY, equal_var=False),XX.mean()/YY.mean()]

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["CAFs"].values
YY=dftype.loc[dftype["TumorCluster"]=="Diff_Neuroblast"]["CAFs"].values
[stats.ttest_ind(XX,YY, equal_var=False),XX.mean()/YY.mean()]

In [ ]:
from scipy.stats import mannwhitneyu


In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["CAFs"].values
YY=dftype.loc[dftype["TumorCluster"]=="Connecting"]["CAFs"].values
mannwhitneyu(XX,YY)

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["CAFs"].values
YY=dftype.loc[dftype["TumorCluster"]=="NC_prog"]["CAFs"].values
mannwhitneyu(XX,YY)

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["CAFs"].values
YY=dftype.loc[dftype["TumorCluster"]=="Diff_Mes"]["CAFs"].values
mannwhitneyu(XX,YY)

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["CAFs"].values
YY=dftype.loc[dftype["TumorCluster"]=="preNeuroblast"]["CAFs"].values
mannwhitneyu(XX,YY)

In [ ]:
XX=dftype.loc[dftype["TumorCluster"]=="Bridge_SCP"]["CAFs"].values
YY=dftype.loc[dftype["TumorCluster"]=="Diff_Neuroblast"]["CAFs"].values
stats.ttest_ind(XX,YY, equal_var=False)

In [ ]:
cd .

In [ ]:
dftype.to_csv("TumorType_vs_EnvironmentCells_100.tab",sep="\t")

# Tumor Calculatioin

# For a1

In [ ]:
dfposi=pd.read_table("A1spot-spotDistance.tab",index_col=0,header=0,sep="\t")
dfposi.shape

In [ ]:
dfposi.head()

In [ ]:
set(dfprobtumor["Patient"])

In [ ]:
dftumor=dfprobtumor.loc[dfprobtumor["Patient"]=="NBLA1"].T
dftumor

In [ ]:
typelist=[]
for j in range(dftumor.shape[1]):
    dftemp=dftumor[set(dftumor.columns)&set(dfposi.loc[dfposi[dftumor.columns[j]]<=100].index)]
    typelist.append(dftemp.T.groupby(["sigType"]).count().reindex(['Diff_Mes', 'NC_prog', 'Bridge_SCP', 'Connecting', 'Chromaffin',
       'preNeuroblast', 'Diff_Neuroblast']).fillna(0).iloc[:,0].T.values.tolist())

In [ ]:
dftype=pd.DataFrame(typelist)

In [ ]:
dftype.shape

In [ ]:
dftumor.shape

In [ ]:
dftype.columns=['Diff_Mes', 'NC_prog', 'Bridge_SCP', 'Connecting', 'Chromaffin',
       'preNeuroblast', 'Diff_Neuroblast']
dftype.index=dftumor.columns
dftype

In [ ]:
dftype=dftype.join(scpdt.obs["TumorCluster"],how="inner")
#dftype.columns=['Bcells', 'Endothelial', 'Fibroblasts', 'Myeloid', 'Schwann', 'Tcells',
#       'sigType']

In [ ]:
dftype.groupby(["TumorCluster"]).count()

In [ ]:
dftype.groupby(["TumorCluster"]).mean().fillna(0)

In [ ]:
dfTumortypeNBLA1=dftype

# For C1

In [ ]:
dfposi=pd.read_table("C1spot-spotDistance.tab",index_col=0,header=0,sep="\t")
dfposi.shape

In [ ]:
dfposi.head()

In [ ]:
set(dfprobtumor["Patient"])

In [ ]:
dftumor=dfprobtumor.loc[dfprobtumor["Patient"]=="NBLC1"].T
dftumor

In [ ]:
typelist=[]
for j in range(dftumor.shape[1]):
    dftemp=dftumor[set(dftumor.columns)&set(dfposi.loc[dfposi[dftumor.columns[j]]<=100].index)]
    typelist.append(dftemp.T.groupby(["sigType"]).count().reindex(['Diff_Mes', 'NC_prog', 'Bridge_SCP', 'Connecting', 'Chromaffin',
       'preNeuroblast', 'Diff_Neuroblast']).fillna(0).iloc[:,0].T.values.tolist())

In [ ]:
dftype=pd.DataFrame(typelist)

In [ ]:
dftype.shape

In [ ]:
dftumor.shape

In [ ]:
dftype.columns=['Diff_Mes', 'NC_prog', 'Bridge_SCP', 'Connecting', 'Chromaffin',
       'preNeuroblast', 'Diff_Neuroblast']
dftype.index=dftumor.columns
dftype

In [ ]:
dftype=dftype.join(scpdt.obs["TumorCluster"],how="inner")
#dftype.columns=['Bcells', 'Endothelial', 'Fibroblasts', 'Myeloid', 'Schwann', 'Tcells',
#       'sigType']

In [ ]:
dftype.groupby(["TumorCluster"]).count()

In [ ]:
dftype.groupby(["TumorCluster"]).mean().fillna(0)

In [ ]:
dfTumortypeNBLC1=dftype

# For A254

In [ ]:
dfposi=pd.read_table("A254_spot-spotDistance.tab",index_col=0,header=0,sep="\t")
dfposi.shape

In [ ]:
dfposi.head()

In [ ]:
set(dfprobtumor["Patient"])

In [ ]:
dftumor=dfprobtumor.loc[dfprobtumor["Patient"]=="A254"].T
dftumor

In [ ]:
typelist=[]
for j in range(dftumor.shape[1]):
    dftemp=dftumor[set(dftumor.columns)&set(dfposi.loc[dfposi[dftumor.columns[j]]<=100].index)]
    typelist.append(dftemp.T.groupby(["sigType"]).count().reindex(['Diff_Mes', 'NC_prog', 'Bridge_SCP', 'Connecting', 'Chromaffin',
       'preNeuroblast', 'Diff_Neuroblast']).fillna(0).iloc[:,0].T.values.tolist())

In [ ]:
dftype=pd.DataFrame(typelist)

In [ ]:
dftype.shape

In [ ]:
dftumor.shape

In [ ]:
dftype.columns=['Diff_Mes', 'NC_prog', 'Bridge_SCP', 'Connecting', 'Chromaffin',
       'preNeuroblast', 'Diff_Neuroblast']
dftype.index=dftumor.columns
dftype

In [ ]:
dftype=dftype.join(scpdt.obs["TumorCluster"],how="inner")
#dftype.columns=['Bcells', 'Endothelial', 'Fibroblasts', 'Myeloid', 'Schwann', 'Tcells',
#       'sigType']

In [ ]:
dftype.groupby(["TumorCluster"]).count()

In [ ]:
dftype.groupby(["TumorCluster"]).mean().fillna(0)

In [ ]:
dfTumortypeA254=dftype

# For B803

In [ ]:
dfposi=pd.read_table("B803_spot-spotDistance.tab",index_col=0,header=0,sep="\t")
dfposi.shape

In [ ]:
dfposi.head()

In [ ]:
set(dfprobtumor["Patient"])

In [ ]:
dftumor=dfprobtumor.loc[dfprobtumor["Patient"]=="B803"].T
dftumor

In [ ]:
typelist=[]
for j in range(dftumor.shape[1]):
    dftemp=dftumor[set(dftumor.columns)&set(dfposi.loc[dfposi[dftumor.columns[j]]<=100].index)]
    typelist.append(dftemp.T.groupby(["sigType"]).count().reindex(['Diff_Mes', 'NC_prog', 'Bridge_SCP', 'Connecting', 'Chromaffin',
       'preNeuroblast', 'Diff_Neuroblast']).fillna(0).iloc[:,0].T.values.tolist())

In [ ]:
dftype=pd.DataFrame(typelist)

In [ ]:
dftype.shape

In [ ]:
dftumor.shape

In [ ]:
dftype.columns=['Diff_Mes', 'NC_prog', 'Bridge_SCP', 'Connecting', 'Chromaffin',
       'preNeuroblast', 'Diff_Neuroblast']
dftype.index=dftumor.columns
dftype

In [ ]:
dftype=dftype.join(scpdt.obs["TumorCluster"],how="inner")
#dftype.columns=['Bcells', 'Endothelial', 'Fibroblasts', 'Myeloid', 'Schwann', 'Tcells',
#       'sigType']

In [ ]:
dftype.groupby(["TumorCluster"]).count()

In [ ]:
dftype.groupby(["TumorCluster"]).mean().fillna(0)

In [ ]:
dfTumortypeB803=dftype

# Merge2

In [ ]:
dftype=dfTumortypeB803.append(dfTumortypeA254).append(dfTumortypeNBLC1).append(dfTumortypeNBLA1)

In [ ]:
scpdt.obs.groupby(['TumorCluster']).count()

In [ ]:
dftype.groupby(["TumorCluster"]).sum()

In [ ]:
dftype.groupby(["TumorCluster"]).count()

In [ ]:
dftype.groupby(["TumorCluster"]).mean()

In [ ]:
8.527473/5.351351

In [ ]:
dftype.groupby(["TumorCluster"]).median()

In [ ]:
dftype.groupby(["TumorCluster"]).max()

In [ ]:
sum(dfposi<=100,1).max()

In [ ]:
cd .

In [ ]:
dftype.to_csv("TumorType_vs_TumorTypeCells.tab",sep="\t")

In [ ]:
sum(dfposi<100).max()

In [ ]:
scpdt

In [ ]:
dftype.head()

# Macrophage

In [ ]:
dftype=pd.read_table("TumorType_vs_EnvironmentCells_100.tab",index_col=0,header=0,sep="\t")

In [ ]:
celltype="Macrophages"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]]["Macrophages"].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]]["Macrophages"].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
(dfmean>1.5)

In [ ]:
sum(dfmean>1.5,1)

In [ ]:
dfp

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.4,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)

# T-cells

In [ ]:
celltype="T-cells"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]][celltype].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]][celltype].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
(dfmean>1.5)

In [ ]:
sum(dfmean>1.5,1)

In [ ]:
dfp

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.6,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)

# B-cells

In [ ]:
celltype="B-cells"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]][celltype].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]][celltype].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
(dfmean>1.5)

In [ ]:
sum(dfmean>1.5,1)

In [ ]:
dfp

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.6,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)

# Endothelial cells

In [ ]:
dftype

In [ ]:
celltype="Endothelial cells"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]][celltype].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]][celltype].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
dfp

In [ ]:
(dfmean>1.5)

In [ ]:
sum(dfmean>1.5,1)

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.6,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)

# CAFs

In [ ]:
celltype="CAFs"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]][celltype].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]][celltype].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
(dfmean>1.5)

In [ ]:
sum(dfmean>1.5,1)

In [ ]:
dfp

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.3,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)

# DCs

In [ ]:
dftype

In [ ]:
celltype="DCs"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]][celltype].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]][celltype].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
(dfmean>1.5)

In [ ]:
sum(dfmean>1.5,1)

In [ ]:
(dfmean>1.5).astype(int)

In [ ]:
dfp

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.6,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)

# Tumor

# Diff_Mes

In [ ]:
cd .

In [ ]:
dftype=pd.read_table("TumorType_vs_TumorTypeCells.tab",index_col=0,header=0,sep="\t")
dftype.head()

In [ ]:
celltype="Diff_Mes"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]][celltype].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]][celltype].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
(dfmean>1.5).astype(int)

In [ ]:
(dfmean>1.5).sum(1)

In [ ]:
dfp

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.6,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)

# NC_prog

In [ ]:
celltype="NC_prog"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]][celltype].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]][celltype].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
(dfmean>1.5).astype(int)

In [ ]:
(dfmean>1.5).sum(1)

In [ ]:
dfp

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.6,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)

# Bridge_SCP

In [ ]:
dftype

In [ ]:
celltype="Bridge_SCP"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]][celltype].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]][celltype].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
(dfmean>1.5).astype(int)

In [ ]:
(dfmean>1.5).sum(1)

In [ ]:
dfp

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.6,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)

# Connecting

In [ ]:
celltype="Connecting"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]][celltype].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]][celltype].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
(dfmean>1.5).astype(int)

In [ ]:
(dfmean>1.5).sum(1)

In [ ]:
dfp

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.6,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)

# preNeuroblast

In [ ]:
celltype="preNeuroblast"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]][celltype].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]][celltype].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
(dfmean>1.5).astype(int)

In [ ]:
(dfmean>1.5).sum(1)

In [ ]:
dfp

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.6,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)

# Chromaffin

In [ ]:
celltype="Chromaffin"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]][celltype].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]][celltype].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
(dfmean>1.5).astype(int)

In [ ]:
(dfmean>1.5).sum(1)

In [ ]:
dfp

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.6,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)

# Diff_Neuroblasts

In [ ]:
dftype

In [ ]:
celltype="Diff_Neuroblast"
dftemp1=dftype[[celltype,"TumorCluster"]]
typelist=list(set(dftemp1["TumorCluster"]))

In [ ]:
from scipy import stats
plist=[]
slist=[]
meanlist=[]
for i in range(len(typelist)):
    temp1=dftemp1.loc[dftemp1["TumorCluster"]==typelist[i]][celltype].values
    plisttemp=[]
    slisttemp=[]
    meanlisttemp=[]
    for j in range(len(typelist)):
        temp2=dftemp1.loc[dftemp1["TumorCluster"]==typelist[j]][celltype].values
        s,p=stats.ttest_ind(temp1,temp2, equal_var = False)
        plisttemp.append('{:.10f}'.format(p))
        slisttemp.append(s)
        meanlisttemp.append(mean(temp1)/mean(temp2))
    plist.append(plisttemp)
    slist.append(slisttemp)
    meanlist.append(meanlisttemp)
    
dfmean=pd.DataFrame(meanlist)
dfmean.index=typelist
dfmean.columns=typelist
dfp=pd.DataFrame(plist)
dfp.index=typelist
dfp.columns=typelist

In [ ]:
dfmean

In [ ]:
(dfmean>1.5).astype(int)

In [ ]:
(dfmean>1.5).sum(1)

In [ ]:
dfp

In [ ]:
mcolor_dict = pd.Series({  
 'SCP':   (90,  10,  10), 'Chromaffin1': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'Sympath':    (170, 180, 170),'SCP':   (190,  10,  10),  'Chromaffin': (25, 60,  130),
                             'cycling':    (57, 105,   7),
             'sympathoblasts':    (170, 180, 170),
                            'Diff_Glia': (235, 125,  128),  'Diff_Mes': (10,10,255),
                          'Chromaffin':   (55, 55,  78), 'preNeuroblast':  (100, 100, 240),
                           'Diff_Neuroblast':    (57, 105,   7),'preGlia': (175, 175,  28),
            'Bridge_SCP':    (170, 180, 170), 'NC_prog':(249,125,5),
                           'Connecting':    ( 95, 186,  70),  'Auto_prog':    (180, 140, 130),
       'cyclingNeuroblast':  (100, 100, 240), 'Migr_prog': ( 80, 235, 255),  'Mes_prog':(249,125,5),
    
    })
mcolor_dict = mcolor_dict.map(lambda x: list(map(lambda y: y/255., x)))
mwanted_order=[
 'Diff_Mes', 'NC_prog',
 'Bridge_SCP', 'Connecting',
 'preNeuroblast',
'Chromaffin', 'Diff_Neuroblast',
]
dftemp1=dftemp1.loc[dftemp1["TumorCluster"]== 'Diff_Mes'].append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'NC_prog']).append(
    dftemp1.loc[dftemp1["TumorCluster"]== 'Bridge_SCP']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Connecting']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='preNeuroblast']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Chromaffin']).append(
    dftemp1.loc[dftemp1["TumorCluster"]=='Diff_Neuroblast'])

In [ ]:
import seaborn as sns
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    for art in ax.get_children():
         if isinstance(art, PolyCollection):
            art.set_edgecolor((0.6,0.6, 0.6))
#seleColor=mnewcolors[rdmvalue]
fig = plt.figure(figsize=(10,10))
fig, ax = plt.subplots()
fig.set_size_inches(16, 8)
sns.set_style("whitegrid")
ax=sns.violinplot(x="TumorCluster", y=celltype, data=dftemp1,bw=0.6,cut=2, gridsize=50,
               saturation=0.9, width=0.75, palette=mcolor_dict[mwanted_order] ,scale='width', inner= "box")
plt.setp(ax.collections, alpha=.8)
ax.set_ylim(ymin=-0.15)
plt.xticks(rotation=90,horizontalalignment='center', verticalalignment='top',
       position=(0,0), fontsize=35)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(celltype,fontsize=20)
plt.grid(False)